In [1]:
import json

def create_request(request_id, prompt, n=3, temperature=0.7, max_tokens=512):
    """
    Crea un diccionario de solicitud con el formato deseado para un archivo JSONL.

    Args:
        request_id (str): Un identificador único para la solicitud.
        prompt (str): El contenido del mensaje del usuario.
        n (int): El número de respuestas a generar.
        temperature (float): El valor de temperatura para la generación de texto.

    Returns:
        dict: Un diccionario con el formato de solicitud.
    """
    request = {
        "custom_id": str(request_id),  # Asegura que sea un string
        "body": {
            "max_tokens": max_tokens,
            "messages": [{"role": "user", "content": prompt}],
            "n": n,
            "temperature": temperature
        }
    }
    return request

In [2]:
from kdir_src.sota.hyde import HyDE, Promptor
from kdir_src.generators.mistral_ai import MistralGenerator
from kdir_src.models.all_models import load_models, get_query_embeddings

In [3]:
KEY = 'jTrxtC1mqMP1Lj9aMg8fvexSh2xDgZC4'
dataset_name = 'arguana'
promptor = Promptor(dataset_name)
generator = MistralGenerator('mistral-small-2503', KEY, 3)
#encoder_models = load_models()
hyde = HyDE(dataset_name, promptor, generator, None)

In [4]:
from kdir_src.dataset.beir import get_sentences_queries

In [5]:
sentences, queries_ids = get_sentences_queries(dataset_name)
requests = []
for i, query in enumerate(sentences):
    prompt = hyde.prompt(query)
    req = create_request(queries_ids[i], prompt)
    requests.append(req)

  0%|          | 0/8674 [00:00<?, ?it/s]

Procesando queries: 100%|██████████████████████████████████████████████████████| 1406/1406 [00:00<00:00, 2043378.87it/s]


In [6]:
# Nombre del archivo JSONL
output_filename = "requests.jsonl"

# Escribir las solicitudes en el archivo JSONL
with open(output_filename, 'w', encoding='utf-8') as f:
    for req in requests:
        # json.dumps convierte el diccionario en una cadena JSON
        # ensure_ascii=False permite caracteres no ASCII (como tildes)
        f.write(json.dumps(req, ensure_ascii=False) + '\n')

print(f"Archivo '{output_filename}' creado exitosamente.")

Archivo 'requests.jsonl' creado exitosamente.


In [7]:
from mistralai import Mistral
import os

api_key = 'jTrxtC1mqMP1Lj9aMg8fvexSh2xDgZC4'

client = Mistral(api_key=api_key)

batch_data = client.files.upload(
    file={
        "file_name": "requests.jsonl",
        "content": open("requests.jsonl", "rb")
    },
    purpose = "batch"
)

In [8]:
batch_data.id

'd9d6ad4d-785c-4a63-bc05-efbf1783abec'

In [9]:
created_job = client.batch.jobs.create(
    input_files=[batch_data.id],
    model="mistral-small-2503",
    endpoint="/v1/chat/completions",
    metadata={"job_type": "testing_hyde"}
)

SDKError: API error occurred: Status 402
{"detail": "You cannot launch batch jobs this big with your free trial. Reduce the number of steps in your configuration or subscribe via the console."}